<div style="font-size:18pt; padding-top:20px; text-align:center">ЗАДАНИЕ 3. <b>Рекомендательные системы и </b> <span style="font-weight:bold; color:green">Spark MLlib</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru, papulin.ru_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Варианты</a></li>
        <li><a href="#2">Задание</a></li>
    </ol>
</div>

# 1. Вариант

<p>Чтобы узнать свой вариант, введите Вашу фамилию в соответвующее поле ниже и запустите ячейку:</p>

In [1]:
surname = "Дубровин" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  1
Задача № 2:  1


<div class="msg-block msg-warning">
  <p class="msg-text-warn">Варианты могут не совпадать с прошлыми ДЗ</p>
</div>

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Задание</div>
    </div>
</div>

<p><b>Датасеты:</b></p>

Небольшой набор данных: <a href="http://files.grouplens.org/datasets/movielens/ml-latest-small.zip">http://files.grouplens.org/datasets/movielens/ml-latest-small.zip</a><br>
Полный набор: <a href="http://files.grouplens.org/datasets/movielens/ml-latest.zip">http://files.grouplens.org/datasets/movielens/ml-latest.zip</a>

<div class="msg-block msg-warning">
  <div class="msg-text-warn">
      <p style="margin:0px">1. Сначала выполните указанные ниже задачи для небольшого набора данных, потом для полного</p>
      <p style="margin:0px">2. Использовать Dataframe. Исключением может быть задача 2</p>
  </div>
</div>

<p><b>Задача 1.</b> Анализ датасета <b><i>(2 балла)</i></b></p>

<p><b style="color:green;">Вариант 1.</b> Animation, Romance, Documentary</p>
<p><b style="color:green;">Вариант 2.</b> Drama, Comedy, Musical</p>
<p><b style="color:green;">Вариант 3.</b> Thriller, Sci-Fi, Adventure</p>

<div class="msg-block msg-info">
  <p class="msg-text-info">Один фильм может принадлежать разным жанрам</p>
</div>

### Подключение библиотек

In [2]:
import pyspark
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

### Создание окружения

In [3]:
import os
import sys

HOME = os.environ["HOME"]
os.environ["SPARK_HOME"]=HOME+"/Workfiles/Hadoop/spark"
os.environ["PYSPARK_PYTHON"]=HOME+"/.conda/2020.11/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]=HOME+"/.conda/2020.11/bin/python"

In [4]:
conf = pyspark.SparkConf() \
        .setAppName("Assignment3") \
        .set("spark.executor.memory", "8G") \
        .set("spark.driver.memory", "8G") \
        .setMaster("local[4]")

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc    = spark.sparkContext

### Жанры по заданию

In [5]:
task_genres = ["Animation", "Romance", "Documentary"]
task_genres_df = spark.createDataFrame([Row(task_genres=task_genres)])

<p>1. Выведите данные, сопоставляющие жанры и количество фильмов</p>

##### Загрузка данных о фильмах

In [6]:
movies_path = "ml-latest-small/movies.csv"

In [7]:
movies_df = spark.read.load(path=movies_path, format="csv", header=True)
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



##### Разбиение по жанрам

In [8]:
movies_df_splited = movies_df \
    .withColumn("genres_split", F.split("genres", "[|]")) \
    .join(task_genres_df) \
    .withColumn("genres_intersect", F.array_intersect("genres_split", "task_genres")) \
    .select("movieId", "title", F.explode("genres_intersect").alias("genre"))
movies_df_splited.show(5)

+-------+--------------------+---------+
|movieId|               title|    genre|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Animation|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|  Romance|
|      7|      Sabrina (1995)|  Romance|
|     11|American Presiden...|  Romance|
+-------+--------------------+---------+
only showing top 5 rows



##### Аггрегация по жанрам

In [9]:
movies_df_splited \
    .groupBy("genre") \
    .count() \
    .show(5)

+-----------+-----+
|      genre|count|
+-----------+-----+
|    Romance| 1596|
|Documentary|  440|
|  Animation|  611|
+-----------+-----+



##### Загрузка рейтингов

In [10]:
ratings_path = "ml-latest-small/ratings.csv"

In [11]:
ratings_df = spark.read.load(path=ratings_path, format="csv", header=True) \
    .withColumn("rating", F.col("rating").cast(DoubleType())) \
    .withColumn("userId", F.col("userId").cast(IntegerType())) \
    .withColumn("movieId", F.col("movieId").cast(IntegerType()))
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



##### Слияние таблиц рейтингов и фильмов

In [12]:
movies_ratings_df = ratings_df.join(movies_df_splited, on = "movieId")
movies_ratings_df.show(5)

+-------+------+------+----------+----------------+---------+
|movieId|userId|rating| timestamp|           title|    genre|
+-------+------+------+----------+----------------+---------+
|      1|   610|   5.0|1479542900|Toy Story (1995)|Animation|
|      1|   609|   3.0| 847221025|Toy Story (1995)|Animation|
|      1|   608|   2.5|1117408267|Toy Story (1995)|Animation|
|      1|   607|   4.0| 964744033|Toy Story (1995)|Animation|
|      1|   606|   2.5|1349082950|Toy Story (1995)|Animation|
+-------+------+------+----------+----------------+---------+
only showing top 5 rows



##### Аггрегация по рейтингам и идентифкаторам фильмов

In [13]:
movies_ratings_agg = movies_ratings_df \
    .groupBy("movieId", "title", "genre") \
    .agg({"rating": "avg", "movieId": "count"}) \
    .withColumnRenamed("avg(rating)", "avg_rating") \
    .withColumnRenamed("count(movieId)", "count")
movies_ratings_agg.show(5)

+-------+--------------------+---------+------------------+-----+
|movieId|               title|    genre|        avg_rating|count|
+-------+--------------------+---------+------------------+-----+
|    302|Queen Margot (Rei...|  Romance|               3.2|    5|
|    745|Wallace & Gromit:...|Animation|3.8958333333333335|   48|
|   2020|Dangerous Liaison...|  Romance|               4.0|   25|
|   2729|       Lolita (1962)|  Romance|               3.5|    7|
|   4317|Love Potion #9 (1...|  Romance|               2.5|    3|
+-------+--------------------+---------+------------------+-----+
only showing top 5 rows



<p>2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соотвествии с вариантом</p>

In [14]:
for genre in task_genres:
    print(genre)
    movies_ratings_agg \
    .filter("genre = '{}'".format(genre)) \
    .select("title", "count") \
    .orderBy("count", ascending = False) \
    .limit(10) \
    .show()

Animation
+--------------------+-----+
|               title|count|
+--------------------+-----+
|    Toy Story (1995)|  215|
|      Aladdin (1992)|  183|
|Lion King, The (1...|  172|
|        Shrek (2001)|  170|
|Beauty and the Be...|  146|
| Finding Nemo (2003)|  141|
|Monsters, Inc. (2...|  132|
|Incredibles, The ...|  125|
|           Up (2009)|  105|
|       WALL·E (2008)|  104|
+--------------------+-----+

Romance
+--------------------+-----+
|               title|count|
+--------------------+-----+
| Forrest Gump (1994)|  329|
|American Beauty (...|  204|
|    True Lies (1994)|  178|
|        Speed (1994)|  171|
|        Shrek (2001)|  170|
|Beauty and the Be...|  146|
|Groundhog Day (1993)|  143|
|Princess Bride, T...|  142|
|Good Will Hunting...|  141|
|      Titanic (1997)|  140|
+--------------------+-----+

Documentary
+--------------------+-----+
|               title|count|
+--------------------+-----+
|Bowling for Colum...|   58|
|Super Size Me (2004)|   50|
|Fahrenheit

<p>3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соотвествии с вариантом</p>

In [15]:
for genre in task_genres:
    print(genre)
    movies_ratings_agg \
    .filter("genre = '{}' and count >= 10".format(genre)) \
    .select("title", "count") \
    .orderBy("count", ascending = True) \
    .show(10)

Animation
+--------------------+-----+
|               title|count|
+--------------------+-----+
|Rescuers Down Und...|   10|
|Girl Who Leapt Th...|   10|
| Mary and Max (2009)|   10|
|Futurama: Bender'...|   10|
|American Tail: Fi...|   10|
|Watership Down (1...|   10|
|        Moana (2016)|   10|
|Final Fantasy VII...|   10|
|    Bee Movie (2007)|   10|
|Polar Express, Th...|   11|
+--------------------+-----+
only showing top 10 rows

Romance
+--------------------+-----+
|               title|count|
+--------------------+-----+
|Open Your Eyes (A...|   10|
|        Focus (2015)|   10|
|Sweetest Thing, T...|   10|
|Nick and Norah's ...|   10|
|Majestic, The (2001)|   10|
|   Trainwreck (2015)|   10|
|Beautiful Girls (...|   10|
|Some Kind of Wond...|   10|
|        Loser (2000)|   10|
| One Fine Day (1996)|   10|
+--------------------+-----+
only showing top 10 rows

Documentary
+--------------------+-----+
|               title|count|
+--------------------+-----+
|Corporation, The .

<p>4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соотвествии с вариантом</p>

In [16]:
for genre in task_genres:
    print(genre)
    movies_ratings_agg \
    .filter("genre = '{}' and count >= 10".format(genre)) \
    .select("title", "count", "avg_rating") \
    .orderBy("avg_rating", ascending = False) \
    .show(10)

Animation
+--------------------+-----+-----------------+
|               title|count|       avg_rating|
+--------------------+-----+-----------------+
|Creature Comforts...|   12|             4.25|
| Mary and Max (2009)|   10|              4.2|
|   Persepolis (2007)|   11|4.181818181818182|
|Grave of the Fire...|   16|          4.15625|
|Spirited Away (Se...|   87|4.155172413793103|
|Ghost in the Shel...|   27|4.148148148148148|
|Batman: Mask of t...|   13|4.115384615384615|
|  Toy Story 3 (2010)|   55|4.109090909090909|
|Girl Who Leapt Th...|   10|              4.1|
|Wallace & Gromit:...|   27|4.092592592592593|
+--------------------+-----+-----------------+
only showing top 10 rows

Romance
+--------------------+-----+------------------+
|               title|count|        avg_rating|
+--------------------+-----+------------------+
|His Girl Friday (...|   14| 4.392857142857143|
|Sunset Blvd. (a.k...|   27| 4.333333333333333|
|It Happened One N...|   14| 4.321428571428571|
|Philadelp

<p>5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соотвествии с вариантом</p>

In [17]:
for genre in task_genres:
    print(genre)
    movies_ratings_agg \
    .filter("genre = '{}' and count >= 10".format(genre)) \
    .select("title", "count", "avg_rating") \
    .orderBy("avg_rating", ascending = True) \
    .show(10)

Animation
+--------------------+-----+------------------+
|               title|count|        avg_rating|
+--------------------+-----+------------------+
|   Shark Tale (2004)|   13|2.3461538461538463|
|Ice Age: Dawn of ...|   14| 2.607142857142857|
|   Happy Feet (2006)|   11|2.6818181818181817|
|American Tail: Fi...|   10|               2.7|
|All Dogs Go to He...|   15|               2.7|
|    Space Jam (1996)|   53| 2.707547169811321|
|Pete's Dragon (1977)|   15|2.7666666666666666|
|Small Soldiers (1...|   18|2.8333333333333335|
|Beavis and Butt-H...|   31| 2.935483870967742|
|Goofy Movie, A (1...|   17|               3.0|
+--------------------+-----+------------------+
only showing top 10 rows

Romance
+--------------------+-----+------------------+
|               title|count|        avg_rating|
+--------------------+-----+------------------+
|Speed 2: Cruise C...|   19| 1.605263157894737|
|     Grease 2 (1982)|   19|2.0789473684210527|
|Heartbreakers (2001)|   10|              2.

<p><b>Задача 2.</b> Коллаборативная фильтрация <b><i>(5 баллов)</i></b></p>

<p><b style="color:green;">Вариант 1.</b> По схожести пользователей</p>
<p><b style="color:green;">Вариант 2.</b> По схожести объектов</p>

<p>6. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга</p>

##### Выбор только нужных данных

In [18]:
df_features = movies_ratings_df.select("userId", "movieId", "rating").distinct()
df_features.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   474|      1|   4.0|
|   458|     48|   3.0|
|    84|    140|   4.0|
|   160|    239|   1.0|
|   217|    351|   2.0|
+------+-------+------+
only showing top 5 rows



##### Разделение

In [19]:
df_train, df_test_init = df_features.randomSplit([0.8, 0.2], seed=123)
df_train.count(), df_test_init.count()

(20104, 5071)

In [20]:
df_train.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     3|   2080|   0.5|
|     4|    708|   4.0|
|    11|   1784|   5.0|
|    12|   1721|   5.0|
|    18|   7147|   4.5|
+------+-------+------+
only showing top 5 rows



<div class="msg-block msg-warning">
  <p><b>К 3 задаче</b></p>
  <p class="msg-text-warn">Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то rmse будет NaN. Поэтому заранее уберите из тестового/валидационного подмножества такие элементы</p>
</div>

In [21]:
df_except = df_test_init.join(df_train, on = "movieId", how = "left_anti")
df_test = df_test_init.exceptAll(df_except.select("userId", "movieId", "rating"))
df_except.count(), df_test.count()

(229, 4842)

In [22]:
df_train.persist().count(), df_test.persist().count()

(20104, 4842)

##### Средний рейтинг

In [23]:
mean_rating_df = df_train.select(F.mean("rating").alias("prediction"))
mean_rating_df.show()

+------------------+
|        prediction|
+------------------+
|3.5455879426979706|
+------------------+



##### Тестирование

In [24]:
df_test_pred_bl = df_test.join(mean_rating_df)
df_test_pred_bl.show(5)

+------+-------+------+------------------+
|userId|movieId|rating|        prediction|
+------+-------+------+------------------+
|    10|   8961|   2.5|3.5455879426979706|
|    19|    745|   4.0|3.5455879426979706|
|    41| 100714|   1.5|3.5455879426979706|
|    51|   2802|   3.0|3.5455879426979706|
|    68|  34162|   5.0|3.5455879426979706|
+------+-------+------+------------------+
only showing top 5 rows



##### Расчёт СКО

In [25]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
eval_rmse.evaluate(df_test_pred_bl)

0.9987840844118598

<p>7. Реализуйте коллаборативную фильтрацию в соотвествии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test</p>

In [26]:
from userrecom import UserBasedRecommend

In [27]:
%load_ext autoreload
%autoreload 2

In [28]:
user_based = UserBasedRecommend(spark)

##### Обучение

In [59]:
user_based.train(df_train, top_N=120, user_column_name="userId", item_column_name="movieId")

##### Получение оценок рейтингов на тестовом подмножестве

In [60]:
df_predict_ub = user_based.predict(df_test, user_column_name="userId", item_column_name="movieId",
                                    predict_column_name = "prediction")
df_predict_ub.show(5)

+------+-------+------+------------------+
|userId|movieId|rating|        prediction|
+------+-------+------+------------------+
|    62|   8360|   1.0| 3.607201598494729|
|   232|    594|   2.5|3.4429862798649795|
|   232|   1029|   2.0|3.3663973768140205|
|   438|   5218|   4.5|3.6701628472689065|
|   543|   2858|   5.0|3.9075017898448965|
+------+-------+------+------------------+
only showing top 5 rows



<p>8. Определите rmse для тестового подмножества</p>

In [31]:
eval_rmse.evaluate(df_predict_ub)

0.9597141022611608

<p><b>Задача 3.</b> Факторизация матрицы <b><i>(3 балла)</i></b></p>

<p>9. Выберите модель ALS по минимальному значению rmse. Для этого используйте kfolds c k=4</p>

##### Параметры

In [32]:
rank = [5, 10, 15] # количество факторов
regParam = [0.001, 0.01, 0.1, 1, 10] # регуляризация

##### Исходный ALS

In [50]:
als_inst = ALS(maxIter=10, seed=123, 
               numUserBlocks=10, numItemBlocks=10, 
               userCol="userId", itemCol="movieId", ratingCol="rating",
               nonnegative=True, coldStartStrategy='drop')

##### Сетка параметров

In [51]:
grid = ParamGridBuilder() \
    .addGrid(als_inst.rank, rank) \
    .addGrid(als_inst.regParam, regParam) \
    .build()

##### Кросс-валидатор

In [52]:
cv = CrossValidator(estimator=als_inst, estimatorParamMaps=grid, evaluator=eval_rmse, parallelism=4, numFolds=4)

##### Обучение моделей

In [53]:
cvModel = cv.fit(df_test)
cvModel.getNumFolds()

4

In [54]:
cvModel.avgMetrics

[3.4294003305623586,
 1.6908769619399466,
 1.171729262704551,
 1.3659312834004116,
 3.7127695589133287,
 3.0333431612161403,
 1.5595790966542287,
 1.1450811190924681,
 1.3661074762713703,
 3.7127695589133287,
 2.1937214305280532,
 1.491242576773981,
 1.1490014755530717,
 1.3668735002433503,
 3.7127695589133287]

##### Результаты на учебном множестве

In [55]:
df_train_pred = cvModel.transform(df_train)
df_train_pred.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   463|   1088|   3.5| 2.6854527|
|   159|   1088|   4.0| 2.8485475|
|   606|   1088|   3.0| 2.3215234|
|   599|   1088|   2.5| 1.6096076|
|   111|   1088|   3.0| 2.5691488|
+------+-------+------+----------+
only showing top 5 rows



##### Резульаты на тестовом множестве

In [56]:
df_predict_asl = cvModel.transform(df_test)
df_predict_asl.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    47|   1088|   4.0|  3.706408|
|   474|   1088|   3.5| 3.2594187|
|   387|   1088|   1.5| 1.8490163|
|   555|   1088|   4.0| 3.8986096|
|   391|   1088|   1.0| 1.4290633|
+------+-------+------+----------+
only showing top 5 rows



<p>10. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов</p>

In [57]:
print( "СКО рекомендаций User-based коллаборативной фильтрации {}".format(eval_rmse.evaluate(df_predict_ub)) )
print( "СКО рекомендаций ASL {}".format(eval_rmse.evaluate(df_predict_asl)) )

СКО рекомендаций User-based коллаборативной фильтрации 0.9597141022611608
СКО рекомендаций ASL 0.2331968100133474


##### Сравнение резульатов

In [58]:
main_clmns = ["userId", "movieId", "rating"]
ub_clmns = main_clmns + [F.col("prediction").alias("pred User-based")]
asl_clmns = main_clmns + [F.col("prediction").alias("pred ASL")]

df_predict_ub.select(ub_clmns) \
.join(df_predict_asl.select(asl_clmns), on = main_clmns) \
.show()

+------+-------+------+------------------+---------+
|userId|movieId|rating|   pred User-based| pred ASL|
+------+-------+------+------------------+---------+
|    47|   1088|   4.0|3.4395312311976913| 3.706408|
|   474|   1088|   3.5|  3.36604112408064|3.2594187|
|   387|   1088|   1.5|3.3101509218038907|1.8490163|
|   555|   1088|   4.0|3.3800596108951666|3.8986096|
|   391|   1088|   1.0|3.3689896085427677|1.4290633|
|   525|   1088|   4.5|3.3844602577946983|3.8024056|
|    42|   1088|   3.0| 3.396628493509407| 3.201251|
|   597|   1959|   4.0| 3.507404759419041|3.9291136|
|   474|   1959|   4.0| 3.522995588492769|  3.79866|
|   138|   1959|   3.5| 3.538793310540483|3.5843968|
|   414|   4519|   2.0|3.3635581164624786| 1.967437|
|   345|   6658|   4.5|2.5529889900993217|4.4154477|
|   474|   8638|   4.0| 3.743872022342787|3.8528438|
|   221|   8638|   4.5|3.7129950258827256| 4.209505|
|   200|   8638|   1.0|3.8213551047292325| 1.297141|
|   388|  44022|   4.5|3.3136061046833607|4.33

<hr>
<div style="width:100%;"><p style="width:100%; text-align:center; font-weight:bold; font-size:18px;">FINLAND</p></div>